In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.ar_model import AR
from xgboost import XGBRegressor
from datetime import datetime
from datetime import timedelta
import common
%matplotlib inline

/Users/venkatesh/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [9]:
def generate_demand_submission():
    train = pd.read_csv('historical_demand.csv')
    test = pd.read_csv('sample_submission/demand_forecast.csv')
    train['pid'] = train.Product_ID.apply(lambda x:int(x[1:]))
    train['rid'] = train.Region.apply(lambda x:int(x[1:]))
    test['pid'] = test.Product_ID.apply(lambda x:int(x[1:]))
    test['rid'] = test.Region.apply(lambda x:int(x[1:]))
    train = train.sort_values(['Month','pid','rid'])
    test = test.sort_values(['Month','pid','rid'])
    combined = pd.concat([train,test],axis=0)
    combined['d3'] = combined.Demand.shift(3*18*81)
    combined['d4'] = combined.Demand.shift(4*18*81)
    combined['d5'] = combined.Demand.shift(5*18*81)
    combined['d6'] = combined.Demand.shift(6*18*81)

    train = combined.iloc[:len(train)].copy()
    test = combined.iloc[len(train):].copy()
    exclude_features=['Product_ID','Region','Demand','Month']
    model = XGBRegressor()
    #model = LinearRegression()
    vtrain = train
    #vtrain = train.loc[train.d13.notnull()]
    model.fit(vtrain[list(set(vtrain.columns)-set(exclude_features))],vtrain.Demand)
    pred = model.predict(test[list(set(test.columns)-set(exclude_features))])
    test['Demand'] = np.floor(pred)
    extra_columns = set(test.columns)-{'Product_ID','Region','Month','Demand'}
    test.drop(extra_columns,inplace=True,axis=1)
    test.sort_values(['Product_ID','Region','Month'])
    test.to_csv('demand_forecast.csv',index=False)
    print("Done creating demand_forecast.csv")
generate_demand_submission()

Done creating demand_forecast.csv


In [10]:
def prune_low_quantities(minimum):
    dem_for = pd.read_csv('demand_forecast.csv')
    old_demand = dem_for.Demand.sum()
    monthly_demand = dem_for.groupby(['Product_ID','Month']).Demand.sum()
    low_monthly_demand = monthly_demand[(monthly_demand > 0) & (monthly_demand < minimum)]
    low_monthly_demand = pd.DataFrame(low_monthly_demand)
    low_monthly_demand.reset_index(inplace=True)
    for index,row in low_monthly_demand.iterrows():
        dem_for.loc[(dem_for.Product_ID==row.Product_ID) &(dem_for.Month==row.Month),'Demand']=0
    dem_for.to_csv('demand_forecast.csv',index=False)
    new_demand = dem_for.Demand.sum()
    print("Reduced demand from ",old_demand,"to",new_demand,(old_demand-new_demand)*100/old_demand,"%")

In [12]:
prune_low_quantities(100)

Reduced demand from  66891.0 to 66315.0 0.8611023904561151 %
